In [1]:
min_distance = 0.7

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import pandas as pd
node_path = '/content/drive/MyDrive/node_classification/text_classification'
df_train = pd.read_pickle(f"{node_path}/data/imdb_embs_train.pkl")
df_val = pd.read_pickle(f"{node_path}/data/imdb_embs_val.pkl")
df_test = pd.read_pickle(f"{node_path}/data/imdb_embs_test.pkl")

In [4]:
import numpy as np
normalized_v = lambda x: x / np.sqrt(np.sum(x**2))

In [5]:
df_train['embedding_normalized'] = df_train['embedding'].apply(lambda x: normalized_v(x))
df_val['embedding_normalized'] = df_val['embedding'].apply(lambda x: normalized_v(x))
df_test['embedding_normalized'] = df_test['embedding'].apply(lambda x: normalized_v(x))

In [6]:
df_train['split'] = 'train'
df_val['split'] = 'val'
df_test['split'] = 'test'
df_total = pd.concat([df_train, df_val, df_test])
df_total['id'] = [i for i in range(len(df_total))]
df_total.index = df_total.id

In [7]:
import numpy as np
all_embs_array = np.stack(df_total['embedding_normalized'].values)

In [8]:
all_ids = list(df_total['id'].values)

In [9]:
list_distances = []

In [10]:
from tqdm.notebook import tqdm
batch_size = 64
list_edges = []
list_distances = []
for i in tqdm(range(0, len(all_embs_array), batch_size), total=len(all_embs_array)//batch_size + 1):
  cur_embs_batch = all_embs_array[i:i+batch_size]
  array_distances = np.dot(cur_embs_batch, all_embs_array.T)
  (distances_indices_source, distances_indices_target) = np.where(array_distances > min_distance)
  for idx_source, idx_target in zip(distances_indices_source, distances_indices_target):
    true_source_idx = all_ids[i + idx_source]
    true_target_idx = all_ids[idx_target]
    if true_source_idx != true_target_idx:
      list_edges.append((true_source_idx, true_target_idx))
      list_distances.append(array_distances[idx_source][idx_target])

  0%|          | 0/782 [00:00<?, ?it/s]

In [11]:
dict_edges = pd.DataFrame({
    'edges' : list_edges,
    'cossine_similarity' : list_distances,
})

In [12]:
df_total.to_pickle(f"{node_path}/data/total_data.pkl")

In [13]:
dict_edges.to_pickle(f"{node_path}/data/edges_data.pkl")